<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/ewha_regulation/one_shot_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**패키지 설치**

In [ ]:
!pip3 install -qU python-dotenv langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
!pip3 install faiss-cpu

**upstage api**

In [ ]:
!pip install langchain-classic

In [ ]:
!pip install openai

In [ ]:
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_classic.chains import RetrievalQA
from langchain_upstage import UpstageDocumentParseLoader
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_core.prompts import PromptTemplate
import re
from openai import OpenAI
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.vectorstores import FAISS
import json

In [ ]:
# set parameter
UPSTAGE_API_KEY = "up_6QotiShLoYSOQBmusnDxusPel5Fuh"
data_path = "/content/drive/MyDrive/assignment/25-2/NLP/ewha.pdf" # folder path containing ewah.pdf and samples.csv

**ewha.pdf loading**
<br>
HTML text로 로드
<br>
기존 MD 텍스트로 로드하였으나 부칙이 일부만 로드되고 잘림. HTML은 전체가 잘 로드됨

In [ ]:
loader = UpstageDocumentParseLoader(
    data_path,
    api_key=UPSTAGE_API_KEY
)

docs = loader.load()

In [ ]:
clean_texts = []

for doc in docs:
    text = doc.page_content

    text = text.strip()
    clean_texts.append(text)


full_text = "\n".join(clean_texts)

부칙과 메인 조항 분리

In [ ]:
match = re.search(r"부칙", full_text)

if not match:
        raise ValueError("부칙 문구를 찾을 수 없습니다.")

addendum_start = match.start()

main_text = text[:addendum_start]
extra_text = text[addendum_start:]

In [ ]:
extra_text[:300]

"부칙(1946. 8. 15 제정)<br>본 학칙은 1946년 3월 1일부터 시행한다.</p><p id='101' data-category='paragraph' style='font-size:18px'>부칙</p><br><p id='102' data-category='list' style='font-size:18px'>① 본 학칙은 1961년 3월 1일부터 시행한다.<br>② 본 학칙 시행에 관한 세칙은 총장이 정한다.</p><p id='103' data-category='paragraph' style='font-size:18px'>"

In [ ]:
def extract_main_text(html_text):
    # 테이블 삭제
    cleaned = re.sub(r'<table.*?>.*?</table>', '', html_text, flags=re.DOTALL)
    return cleaned

In [ ]:
cleaned_extra_text = extract_main_text(extra_text)
print(cleaned_extra_text)

부칙(1946. 8. 15 제정)<br>본 학칙은 1946년 3월 1일부터 시행한다.</p><p id='101' data-category='paragraph' style='font-size:18px'>부칙</p><br><p id='102' data-category='list' style='font-size:18px'>① 본 학칙은 1961년 3월 1일부터 시행한다.<br>② 본 학칙 시행에 관한 세칙은 총장이 정한다.</p><p id='103' data-category='paragraph' style='font-size:18px'>부칙(문관행 1040.1-1549, 1963. 12. 16 개정)<br>본 학칙은 1964년 3월 1일부터 시행한다.</p><p id='104' data-category='paragraph' style='font-size:18px'>부칙(문고대 1041.3-16, 1965. 1. 5 개정)</p><br><p id='105' data-category='list' style='font-size:18px'>ⓛ 이 학칙은 1965년 1월 10일부터 시행한다.<br>② 이 학칙 시행 당시 문리과대학에 소속하는 가정학과와 사범대학에 소속하는 가정학과는<br>폐과하되 그 재적 학생의 졸업년도까지 존치한다.</p><p id='106' data-category='paragraph' style='font-size:18px'>부칙(문고대 1041.3-89, 1966. 2. 1 개정)</p><p id='107' data-category='paragraph' style='font-size:18px'>2 - 2 - 17</p><h1 id='108' style='font-size:14px'>이화여자대학교 학칙</h1><h1 id='109' style='font-size:18px'>본 학칙은 1966년 2월 1일부터 시행한다.</h1><p id='110' data-category='paragraph' style='font-size:18px'>부칙(문고대 1041

# MD format

In [ ]:
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)

## 메인 조항 formatting

In [ ]:
prompt = """
You are an expert in Korean legal document analysis and Markdown formatting.

Your task:
Convert the given Ewha University regulation texts (in HTML text) into a clean Markdown document with correctly assigned heading levels.
You are given a main text:
1) MAIN TEXT: the main university regulations

Process BOTH texts together and apply the same rules.

=================================================================
MARKDOWN HEADING RULES
=================================================================

**Main Body Chapters (e.g., “제1장 총칙”)**:
- Top-level header: `# [Chapter Title]`
- Articles within chapters: `## [Article Title]`

=================================================================
ADDITIONAL RULES
=================================================================
- Remove ALL page numbers (e.g., “2-2-1”).
- Remove HTML tags, styles, tables, or formatting artifacts from the PDF conversion.
- Preserve all meaningful text content, but organize it strictly under the specified Markdown headings.
- Do NOT hallucinate any content not present in the text.
- Do NOT output explanations or any text outside the markdown.
- Output MUST be clean markdown and ONLY markdown.

Below are the two raw texts:

---------------- MAIN TEXT START ----------------
{main_text}
---------------- MAIN TEXT END ----------------

"""


messages = [
    {
        "role": "system",
        "content": "You are an expert in hierarchical legal information extraction and Markdown document structuring."
    },
    {
        "role": "user",
        "content": prompt.format(main_text=main_text)
    }
]

md_response = client.chat.completions.create(
    model="solar-pro2",
    messages=messages
    # response_format=response_format
)

In [ ]:
print(md_response.choices[0].message.content[:500])

# 이화여자대학교 학칙

## 제1장 총칙

### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과  
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에  
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.

### 제2조(명칭)
본교는 이화여자대학교라 부른다.

### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)

## 제2장 편제

### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.  
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범  
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각  
대학”이라 한다) (개정 2016.6.16.)  
2. 호크마(HOKMA)교양대학  
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문


## 부칙 formatting

In [ ]:
extra_prompt = """
You are an expert in Korean legal document analysis and Markdown formatting.

Your task:
Convert the given text into a clean Markdown document with correctly assigned heading levels.

=================================================================
MARKDOWN HEADING RULES
=================================================================

**Main Body Chapters (e.g., “제1장 총칙”)**:
- Top-level header: `# [Chapter Title]`
- Articles within chapters: `## [Article Title]`

=================================================================
SPECIFIC INSTRUCTIONS FOR THIS TEXT
=================================================================
- The overall title for this section should be `# 이화여자대학교 학칙`. This should be the first line of the output.
- Every instance of '부칙' that introduces a new addendum should be formatted as a second-level Markdown header, '## 부칙'. Ensure its associated content follows immediately under this header.
- Remove ALL page numbers (e.g., “2-2-1”).
- Remove HTML tags, styles, tables, or formatting artifacts from the PDF conversion.
- Preserve all meaningful text content, but organize it strictly under the specified Markdown headings.
- Do NOT hallucinate any content not present in the text.
- Do NOT output explanations or any text outside the markdown.
- Output MUST be clean markdown and ONLY markdown.

Below is the raw text:

---------------- ADDENDUM TEXT START ----------------
{extra_text}
---------------- ADDENDUM TEXT END ----------------
"""

extra_messages = [
    {
        "role": "system",
        "content": "You are an expert in hierarchical legal information extraction and Markdown document structuring."
    },
    {
        "role": "user",
        "content": extra_prompt.format(extra_text=cleaned_extra_text)
    }
]

md_extra_response = client.chat.completions.create(
    model="solar-pro2",
    messages=extra_messages
)

print(md_extra_response.choices[0].message.content[:500])

# 이화여자대학교 학칙

## 부칙(1946. 8. 15 제정)
본 학칙은 1946년 3월 1일부터 시행한다.

## 부칙
① 본 학칙은 1961년 3월 1일부터 시행한다.  
② 본 학칙 시행에 관한 세칙은 총장이 정한다.

## 부칙(문관행 1040.1-1549, 1963. 12. 16 개정)
본 학칙은 1964년 3월 1일부터 시행한다.

## 부칙(문고대 1041.3-16, 1965. 1. 5 개정)
① 이 학칙은 1965년 1월 10일부터 시행한다.  
② 이 학칙 시행 당시 문리과대학에 소속하는 가정학과와 사범대학에 소속하는 가정학과는  
폐과하되 그 재적 학생의 졸업년도까지 존치한다.

## 부칙(문고대 1041.3-89, 1966. 2. 1 개정)
본 학칙은 1966년 2월 1일부터 시행한다.

## 부칙(문고대 1041.3-89, 1966. 2. 1 개정)
본 학칙은 1966년 2월 1일부터 시행한다.

## 부칙(문고대 1041.3-171, 1967. 1. 24 개


### MD Split

### MD 택스트를 ## 장 단위로 split

In [ ]:
headers_to_split_on = [("##", "Header 2")]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

md_header_splits = markdown_splitter.split_text(md_response.choices[0].message.content)
extra_splits = markdown_splitter.split_text(md_extra_response.choices[0].message.content)
md_header_splits.extend(extra_splits)

In [ ]:
for header in md_header_splits[:3]:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

# 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.  
### 제2조(명칭)
본교는 이화여자대학교라 부른다.  
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
{'Header 2': '제1장 총칙'}
## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]  
### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1
5.)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할
전공을 결정

### md를 ### 조 단위로 split

In [ ]:
small_headers_to_split_on = [
    ("##", "Header 2"),
     ("###", "Header 3")
]
small_md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=small_headers_to_split_on, strip_headers=False)
small_headers_splits = small_md_splitter.split_text(md_response.choices[0].message.content)
small_headers_splits.extend(small_md_splitter.split_text(md_extra_response.choices[0].message.content))

In [ ]:
for header in small_headers_splits[:5]:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

# 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.
{'Header 2': '제1장 총칙', 'Header 3': '제1조(목적)'}
### 제2조(명칭)
본교는 이화여자대학교라 부른다.
{'Header 2': '제1장 총칙', 'Header 3': '제2조(명칭)'}
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
{'Header 2': '제1장 총칙', 'Header 3': '제3조(위치)'}
## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]
{'Header 2': '제2장 편제', 'Header 3': '제4조(대학 및 대학원)'}
### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1
5.)
② 모집단위별 입학정원의 일부는 입학전형에 따라

# embedding

In [ ]:
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

# pdf table 추출

In [ ]:
!pip install camelot-py[cv]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 26.5 MB/s eta 0:00:00


In [ ]:
import camelot
import fitz
import pandas as pd

In [ ]:
pdf_path = data_path

# Step 1: lattice 모드로 테이블 bbox만 추출
lattice_tables = camelot.read_pdf(pdf_path, pages="all", flavor="lattice")

final_tables = []

for tbl in lattice_tables:
    page_num = tbl.page
    bbox = tbl._bbox  # (x1, y1, x2, y2)

    # Step 2: PyMuPDF로 페이지 crop
    doc = fitz.open(pdf_path)
    page = doc[page_num - 1]

    rect = fitz.Rect(bbox)
    clipped = page.get_pixmap(clip=rect)
    tmp_img = f"tmp_page_{page_num}.png"
    clipped.save(tmp_img)

    # Step 3: stream 모드로 crop된 이미지 분석
    stream_tbls = camelot.read_pdf(
        pdf_path,
        pages=str(page_num),
        flavor="stream",
        table_areas=[",".join(map(str, bbox))]  # bbox로 영역 제한
    )

    # Step 4: 이 테이블들만 최종 저장
    for s in stream_tbls:
        final_tables.append(s.df)

# 출력 예시
for i, df in enumerate(final_tables):
    # print(f"=== Final Table {i+1} ===")
    # print(df.to_markdown(index=False))
    df.to_csv(f'table{i}.csv', index=False)
    print(f'Table {i} extracted')

Table 0 extracted
Table 1 extracted
Table 2 extracted
Table 3 extracted
Table 4 extracted
Table 5 extracted
Table 6 extracted
Table 7 extracted
Table 8 extracted
Table 9 extracted
Table 10 extracted
Table 11 extracted
Table 12 extracted
Table 13 extracted
Table 14 extracted
Table 15 extracted
Table 16 extracted
Table 17 extracted


In [ ]:
cleaned_df = []

for _, df in enumerate(final_tables):
  headers_raw = [str(item).strip().replace(' ', '').replace('\n', '').replace('\u3000', '') for item in df.iloc[1]]

  for col in df.columns:
      df[col] = df[col].astype(str).str.strip().str.replace(' ', '').str.replace('\n', '').str.replace('\u3000', '')
      df[col] = df[col].ffill()
      cleaned_df.append(df)

for df in cleaned_df[:3]:
  print(df)
  print('==========================')

         0      1         2
0       대학  학위의종류    학과또는전공
1   사회과학대학    문학사     사회복지학
2   자연과학대학    이학사    분자생명과학
3     공과대학    공학사      컴퓨터학
4                     정보통신학
5                  건축학(4년제)
6            건축학사  건축학(5년제)
7             공학사       환경학
8     음악대학   음악학사      건반악기
9                       관현악
10                       성악
11                     교회음악
12                       작곡
13                     한국음악
14  조형예술대학   미술학사       한국화
15                    회화․판화
16                       조소
17                    환경디자인
18                  시각정보디자인
19                    산업디자인
20                    패션디자인
21                     섬유예술
22                     도자예술
23  체육과학대학    이학사       체육학
24                    사회체육학
         0      1         2
0       대학  학위의종류    학과또는전공
1   사회과학대학    문학사     사회복지학
2   자연과학대학    이학사    분자생명과학
3     공과대학    공학사      컴퓨터학
4                     정보통신학
5                  건축학(4년제)
6            건축학사  건축학(5년제)
7             공학사       환경학
8     음악대학   음악학사   

In [ ]:
from langchain_core.documents import Document

In [ ]:
table_documents = []

for i, df in enumerate(cleaned_df):
  table_doc = Document(page_content=df.to_markdown(index=False),
                       metadata={'Header 2': '별표'+str(i)})
  table_documents.append(table_doc)

In [ ]:
for doc in table_documents:
  print(doc.page_content)
  print(doc.metadata)
  print('*********************************************')

| 0            | 1          | 2              |
|:-------------|:-----------|:---------------|
| 대학         | 학위의종류 | 학과또는전공   |
| 사회과학대학 | 문학사     | 사회복지학     |
| 자연과학대학 | 이학사     | 분자생명과학   |
| 공과대학     | 공학사     | 컴퓨터학       |
|              |            | 정보통신학     |
|              |            | 건축학(4년제)  |
|              | 건축학사   | 건축학(5년제)  |
|              | 공학사     | 환경학         |
| 음악대학     | 음악학사   | 건반악기       |
|              |            | 관현악         |
|              |            | 성악           |
|              |            | 교회음악       |
|              |            | 작곡           |
|              |            | 한국음악       |
| 조형예술대학 | 미술학사   | 한국화         |
|              |            | 회화․판화      |
|              |            | 조소           |
|              |            | 환경디자인     |
|              |            | 시각정보디자인 |
|              |            | 산업디자인     |
|              |            | 패션디자인     |
|              |            | 섬유예술       |
|              |      

# vector store에 저장

### text로 simple vectorstore

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

In [ ]:
doc = text_splitter.create_documents([full_text])

In [ ]:
simple_vectorstore = FAISS.from_documents(doc, upstage_embeddings)

### 큰 chunk vectorstore

In [ ]:
# 장 내부에 텍스트가 너무 긴 경우 (chunk size 1500 이상) 추가 split
fine_grained_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=100
)

In [ ]:
big_docs = []

# 메인 조항 % 부칙 필요 시 split 후 append
for doc in md_header_splits:
    sub_chunks_content = fine_grained_splitter.split_text(doc.page_content)
    for chunk_content in sub_chunks_content:
        # 기존 metadata 유지
        new_metadata = doc.metadata.copy()
        big_docs.append(Document(page_content=chunk_content, metadata=new_metadata))

for table in table_documents:
  big_docs.append(Document(page_content=table.page_content, metadata=table.metadata))

In [ ]:
# Now create the vector store from the fine_grained_documents
big_vectorstore = FAISS.from_documents(big_docs, upstage_embeddings)

In [ ]:
for doc in big_docs[:3]:
    print(doc.page_content)
    print(doc.metadata)
    print('----------------------')

# 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.  
### 제2조(명칭)
본교는 이화여자대학교라 부른다.  
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
{'Header 2': '제1장 총칙'}
----------------------
## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]  
### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1
5.)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후

### 작은 chunk vectorstore

In [ ]:
small_docs = []

for doc in small_headers_splits:
  new_metadata = doc.metadata.copy()
  small_docs.append(Document(page_content=doc.page_content, metadata=new_metadata))

for table in table_documents:
  small_docs.append(Document(page_content=table.page_content, metadata=table.metadata))

In [ ]:
small_vectorstore = FAISS.from_documents(small_docs, upstage_embeddings)

### Try Parent Document Retriever

In [ ]:
import uuid
from langchain_core.documents import Document
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_core.stores import InMemoryStore

In [ ]:
docstore = InMemoryStore() # K/V store for Big Chunks

all_child_documents = [] # This will replace your original 'final_documents'
parent_docs_to_store = [] # Collects all Big Chunks

# Process md_header_splits (Main Regulations + Extra)
for doc in md_header_splits:
    parent_id = str(uuid.uuid4()) # Generate a unique ID for the parent

    # Store the parent document (Big Chunk)
    docstore.mset([(parent_id, doc)])
    parent_docs_to_store.append(doc) # Add to collection for docstore

    # Split and tag the child documents (Small Chunks)
    # sub_chunks_content = fine_grained_splitter.split_text(doc.page_content)
    for chunk_content in small_headers_splits:
        new_metadata = doc.metadata.copy()
        new_metadata["parent_id"] = parent_id # 💡 CRITICAL: Link to parent
        all_child_documents.append(Document(page_content=chunk_content.page_content, metadata=new_metadata))

In [ ]:
# Process table_documents (Appendices/Tables)
from langchain_text_splitters import RecursiveCharacterTextSplitter
small_table_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100
)

for doc in table_documents:
    parent_id = str(uuid.uuid4()) # Generate a unique ID for the parent

    # Store the parent document (Big Chunk)
    docstore.mset([(parent_id, doc)])
    parent_docs_to_store.append(doc) # Add to collection for docstore

    # Split and tag the child documents (Small Chunks)
    sub_chunks_content = small_table_splitter.split_text(doc.page_content)
    for chunk_content in sub_chunks_content:
        new_metadata = doc.metadata.copy()
        new_metadata["parent_id"] = parent_id # 💡 CRITICAL: Link to parent
        all_child_documents.append(Document(page_content=chunk_content, metadata=new_metadata))

**embedding**

In [ ]:
vectorstore = FAISS.from_documents(
    all_child_documents,
    upstage_embeddings
)

KeyboardInterrupt: 

In [ ]:
# Initialize the Parent Document Retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    # We pre-split all documents, so we pass None for internal splitters
    parent_splitter=None,
    child_splitter=None,
    search_kwargs={"k": 3} # The number of SMALL chunks to retrieve initially
)

print(f"Total Big/Parent Documents stored: {len(parent_docs_to_store)}")
print(f"Total Small/Child Documents indexed in FAISS: {len(all_child_documents)}")
print("ParentDocumentRetriever setup is complete.")

# QA

**upstage llm 불러오기**

In [ ]:
llm = ChatUpstage(api_key = UPSTAGE_API_KEY, model="solar-pro2")

**prompt template**

In [ ]:
prompt_template = PromptTemplate.from_template(
    """
    You are a quiz taker about Ewha Womans University's regulation.
    Please provide most correct answer for the given context in the response format below.

    Response Format:
    [Answer]: (A) answer <OR> (X) The information is not present in the context.
    [Reason]: Brief reason for your answer
    [Final Answer]: THE final answer to the question following your given reason. Answer in this format: (A) answer

    If the answer is not present in the context, please answer as "(X) The information is not present in the context."

    Example:
    QUESTION) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?
    (A) 인정 안됨
    (B) 1학점
    (C) 2학점
    (D) 3학점

    Answer)
    [Answer]: (D) 3학점
    [Reason]: The context explicitly states in 제48조의2(영어 및 정보인증) ①: "영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다." This directly answers the question by specifying that 3 credits are recognized for such certifications.
    [Final Answer]: (D) 3학점

    Here is the question and context:
    ---
    Question: {question}
    ---
    Context: {context}
    """
)

**쿼리 예시**

# testset load

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/assignment/25-2/NLP/testset.csv')

prompts = data['prompts']
answers = data['answers']

ewha_prompts = prompts[:25]
ewha_answers = answers[:25]

In [ ]:
print(len(ewha_prompts))
print(ewha_prompts)

25
0     QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 ...
1     QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 ...
2     QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는...
3     QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?\n(...
4     QUESTION5) 2019학년도 휴먼기계바이오공학부의 입학 정원은 몇 명인가? \...
5     QUESTION6) 1980학년도 이전 입학생에 대하여 적용하는 등급에 따른 성적점...
6     QUESTION7) 사회체육학과 소속 학생에게 수여하는 학위는 무엇인가? \n(A)...
7     QUESTION8) 복수전공 신청 자격에 해당하지 않는 것은? \n(A) 1학년을 ...
8     QUESTION9) 이화여자대학교의 설립 정신은 무엇인가요?  (A) 공산주의 이념...
9     QUESTION10) 이화여자대학교의 위치는 어디인가요?  (A) 강남구  (B) ...
10    QUESTION11) 학점 기준에 따르면 1학점당 수업 시간은 몇 시간 이상이어야 ...
11    QUESTION12) 전공과 관련된 조항으로 올바른 것은?\n(A) 2 이상의 학부...
12    QUESTION13) 등록금에 관한 조항으로 올바르지 않은 것은?\n(A) 총장은 ...
13    QUESTION14) 전공과목은 어떻게 구분되는가?\n(A) 전공기초과목과 전공과목...
14    QUESTION15) 학칙개정에 관한 설명으로 옳지 않은 것은?\n(A) 학칙 개정...
15    QUESTION16)계절학기에 취득할 수 있는 최대 학점은 몇 점입니까?\n(A) ...
16    QUESTION17)졸업하기 위해 총평균 성적이 충족해야 하는 최소 기준은?\n(A...
17    QUESTION18)수업의 결석이 출석으로 인정될 수 있는 사유가 아닌

**response에서 정답만 추출**

In [ ]:
import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are no answers formatted like the format, it returns None.
    """
    pattern = r"\[Final Answer\]:\s\(([A-J])\)[^)]*"

    # Readjustment로 인해 dinal answer가 여러 번 등장할 수 있으므로 findall 사용
    matches = re.findall(pattern, response)

    if matches:
        print(f'Matched answer: {matches}')  # debug
        # 가장 마지막 final answer만 선택
        return matches[-1].strip()
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        print(f'Re-matched answer: {match}')  # debug
        return match.group(0)
    else:
        return None

In [ ]:
def get_responses(qa, prompts):
  responses = []

  for prompt in prompts:
  # for prompt in ewha_prompts:
      response = qa.invoke(prompt)
      # responses.append(response.content)
      print(response['result'])  # debuggiing

      final_answer = extract_answer(response['result'])
      print(final_answer)
      print('************************')
      responses.append(final_answer)

  return responses

**print accuracy**

In [ ]:
def get_accuracy(responses, answers):
  cnt = 0

  for answer, response in zip(answers, responses):
      print("-"*10)
      # generated_answer = extract_answer(response)
      # print(response)
      # check
      if response:
          print(f"generated answer: {response}, answer: {answer}")
      else:
          print(f"extraction fail, answer: {answer}")


      if response == None:
          continue
      if response in answer:
          cnt += 1

  # print()
  # print(f"acc: {(cnt/len(ewha_prompts))*100}%")

  accuracy = (cnt/len(ewha_prompts))*100
  return accuracy

**majority vote**

In [ ]:
import collections

In [ ]:
def majority_voting(qa, prompt):
  majority_voted_answers = []
  num_iterations = 5

  for idx, prompt in enumerate(prompt):
      answers_for_current_prompt = []
      for _ in range(num_iterations):
          response = qa.invoke(prompt)
          final_answer = extract_answer(response['result'])
          if final_answer:
              answers_for_current_prompt.append(final_answer)

      if answers_for_current_prompt:
          answer_counts = collections.Counter(answers_for_current_prompt)
          most_common_answer = answer_counts.most_common(1)[0][0]
          majority_voted_answers.append(most_common_answer)
          print(f"Prompt {idx+1}: Majority voted answer = {most_common_answer}")
      else:
          majority_voted_answers.append(None)
          print(f"Prompt {idx+1}: No valid answers extracted.")

  return majority_voted_answers

# big chunk retriever

In [ ]:
big_retriever = big_vectorstore.as_retriever(search_kwargs={"k": 3}) #top 5 문서반환
big_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=big_retriever,
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 },
                                 return_source_documents=True) # 원문출처받기

# small chunk retriever

In [ ]:
small_retriever = small_vectorstore.as_retriever(search_kwargs={"k": 5}) #top 5 문서반환
small_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=small_retriever,
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 },
                                 return_source_documents=True) # 원문출처받기

# big + small ensemble

In [ ]:
from langchain_classic.retrievers import EnsembleRetriever

In [ ]:
# 두 retriever의 결과를 ensemble
retriever = EnsembleRetriever(
    retrievers=[big_retriever, small_retriever],
    weights=[0.3, 0.7]    # small chunk의 정확도가 더 높을 때 small에 가중치↑
)

ensemble_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=retriever,
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 })

### 5 iter

In [ ]:
iter_answers = []

for i in range(5):
  responses = get_responses(ensemble_qa, ewha_prompts)
  acc = (get_accuracy(responses, ewha_answers))
  print(f'{i}th Accuracy: {acc}%')
  iter_answers.append(acc)

print(f"Avg accuracy(5 Iteration): {sum(iter_answers) / 5}%")

[Answer]: "(D) 90일",  
[Reasoning_Source]: "제26조(휴학) ⑥: '재학 중인 자가 휴학을 하고자 하는 경우 학기개시일로부터 90일 이내에 휴학을 신청하여야 한다.' (90일)",  
[Final_Answer]: "(D) 90일"  

### 해설  
- **제26조(휴학) ⑥항**에 명시적으로 "학기개시일로부터 **90일 이내**에 휴학 신청"이 규정되어 있습니다.  
- 다른 조항(예: 제5장 제13조의 입학시기 30일)은 휴학과 무관하므로 배제합니다.  
- 정답은 **(D) 90일**이며, 근거가 명확히 제시되어 (X)가 아닙니다.
Re-matched answer: <re.Match object; span=(12, 13), match='D'>
D
************************
[Answer]: "(B) 3",
[Reasoning_Source]: "제31조(재입학) ②: '재입학은 1회에 한하여 할 수 있다. 다만, 제28조제4호에 의하여 제적된 자는 제적된 날부터 1년이 경과한 후 재입학할 수 있다.' (a=1, b=1 → a+b=2가 아닌, 문맥상 a=1, b=1로 명시되었으나, 질문에서 a+b 값을 묻는 점과 옵션 간 논리적 불일치로 인해 (B) 3이 아닌 (A) 2가 정답이나, 제공된 옵션 중 (B) 3이 가장 근접함. 단, 정확한 계산 시 a=1, b=1이므로 a+b=2로 (A)가 정답이지만, 예시와 규칙 적용 시 (X)가 아님을 고려해 재분석 필요. 재확인 결과, 문맥상 a=1, b=1로 a+b=2이며, 이는 옵션 (A)에 해당. 그러나 초기 분석에서 (B)를 선택한 오류를 수정함.)"  
*(수정 후 최종 근거)*  
"제31조(재입학) ②에 명시된 '재입학은 1회(a=1)에 한하여 할 수 있다'와 '제28조제4호에 의해 제적된 자는 1년(b=1) 경과 후 재입학 가능' 규정을 종합하면, a+b=2. 따라서 정답은 (A) 2."  

[Final_Answer]: "(A) 2"  

*(최종 답변

### majority voting

In [ ]:
voted_answers = majority_voting(ensemble_qa, ewha_prompts)

Re-matched answer: <re.Match object; span=(12, 13), match='D'>
Re-matched answer: <re.Match object; span=(12, 13), match='D'>
Re-matched answer: <re.Match object; span=(12, 13), match='D'>
Re-matched answer: <re.Match object; span=(12, 13), match='D'>
Re-matched answer: <re.Match object; span=(12, 13), match='D'>
Prompt 1: Majority voted answer = D
Re-matched answer: <re.Match object; span=(12, 13), match='B'>
Re-matched answer: <re.Match object; span=(12, 13), match='B'>
Re-matched answer: <re.Match object; span=(12, 13), match='B'>
Re-matched answer: <re.Match object; span=(12, 13), match='B'>
Re-matched answer: <re.Match object; span=(12, 13), match='B'>
Prompt 2: Majority voted answer = B
Re-matched answer: <re.Match object; span=(12, 13), match='C'>
Re-matched answer: <re.Match object; span=(12, 13), match='C'>
Re-matched answer: <re.Match object; span=(12, 13), match='C'>
Re-matched answer: <re.Match object; span=(12, 13), match='C'>
Re-matched answer: <re.Match object; span=(12,

KeyboardInterrupt: 

In [ ]:
print(f"Major voting accuracy: {get_accuracy(voted_answers, ewha_answers)}%")